In [20]:
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer

In [21]:
from Data_loading_from_excel import load_fish_data

### Loading Data

In [22]:
df = load_fish_data("../Data/Raw/Main_Data.xlsx", sheet_name="Data edited")
pd.set_option('display.max_columns', None)
df.head()

,Date,Juilin Date,Month,Day,Year,row,Week,AM Feed,AM Transparency,PM Feed,PM Transparency,Stocked,Spring Temp (F),Max Spring (F),Min Spring (F),Daily turbi mean,Max daily turbi,Max air temp,Min air temp,Dec Rain,Calmar Rain,Year class,Strain,Lot,Sub Lot,Raceway,# fish,Morts,Fish Alive,Fish survival rate,Sub Lot % morts,Lot % mort,Strain % mort,Comments
0,2022-04-22,44673,4,22,2022,477,17,Y,5.450833,X,5.918333,NaN,46.7325,46.94,46.4,6.154271,9.9,64.0,37.0,0.67,0.61,2021,Arlee,ARA2,ARA2A2&A7A2,A2,10023,NaN,10023,100.0,0.0,0.0,0.0,NaN
1,2022-04-22,44673,4,22,2022,477,17,Y,5.450833,X,5.918333,NaN,46.7325,46.94,46.4,6.154271,9.9,64.0,37.0,0.67,0.61,2021,Shasta,STA3,STA3A3,A3,10010,NaN,10010,100.0,0.0,0.0,0.0,NaN
2,2022-04-22,44673,4,22,2022,477,17,Y,5.450833,X,5.918333,NaN,46.7325,46.94,46.4,6.154271,9.9,64.0,37.0,0.67,0.61,2021,Shasta,STA4,STA4A4,A4,10385,NaN,10385,100.0,0.0,0.0,0.0,NaN
3,2022-04-22,44673,4,22,2022,477,17,Y,5.450833,X,5.918333,NaN,46.7325,46.94,46.4,6.154271,9.9,64.0,37.0,0.67,0.61,2021,Shasta,STA4,STA4A5,A5,10595,NaN,10595,100.0,0.0,0.0,0.0,NaN
4,2022-04-22,44673,4,22,2022,477,17,Y,5.450833,X,5.918333,NaN,46.7325,46.94,46.4,6.154271,9.9,64.0,37.0,0.67,0.61,2021,Shasta,STA6,STA6A6,A6,7806,NaN,7806,100.0,0.0,0.0,0.0,NaN


In [23]:
selected_columns = ["Date", "Month","Day", "Year", "AM Feed", "AM Transparency", "PM Feed", "PM Transparency","Stocked", "Spring Temp (F)", "Morts", "# fish", "Dec Rain", "Calmar Rain", "Strain", "Lot", "Sub Lot", "Raceway", "# fish","Morts", "Fish Alive", "Fish survival rate"]
new_df = df[selected_columns]
new_df.head()

,Date,Month,Day,Year,AM Feed,AM Transparency,PM Feed,PM Transparency,Stocked,Spring Temp (F),Morts,# fish,Dec Rain,Calmar Rain,Strain,Lot,Sub Lot,Raceway,# fish,Morts,Fish Alive,Fish survival rate
0,2022-04-22,4,22,2022,Y,5.450833,X,5.918333,NaN,46.7325,NaN,10023,0.67,0.61,Arlee,ARA2,ARA2A2&A7A2,A2,10023,NaN,10023,100.0
1,2022-04-22,4,22,2022,Y,5.450833,X,5.918333,NaN,46.7325,NaN,10010,0.67,0.61,Shasta,STA3,STA3A3,A3,10010,NaN,10010,100.0
2,2022-04-22,4,22,2022,Y,5.450833,X,5.918333,NaN,46.7325,NaN,10385,0.67,0.61,Shasta,STA4,STA4A4,A4,10385,NaN,10385,100.0
3,2022-04-22,4,22,2022,Y,5.450833,X,5.918333,NaN,46.7325,NaN,10595,0.67,0.61,Shasta,STA4,STA4A5,A5,10595,NaN,10595,100.0
4,2022-04-22,4,22,2022,Y,5.450833,X,5.918333,NaN,46.7325,NaN,7806,0.67,0.61,Shasta,STA6,STA6A6,A6,7806,NaN,7806,100.0


### AM Feed and PM Feed can be imputed by 0s and 1s

In [24]:
new_df.loc[:, "AM Feed"] = new_df["AM Feed"].apply(lambda x: 1 if x == "Y" else 0)
new_df.loc[:, "PM Feed"] = new_df["PM Feed"].apply(lambda x: 1 if x == "Y" else 0)

new_df.head()

,Date,Month,Day,Year,AM Feed,AM Transparency,PM Feed,PM Transparency,Stocked,Spring Temp (F),Morts,# fish,Dec Rain,Calmar Rain,Strain,Lot,Sub Lot,Raceway,# fish,Morts,Fish Alive,Fish survival rate
0,2022-04-22,4,22,2022,1,5.450833,0,5.918333,NaN,46.7325,NaN,10023,0.67,0.61,Arlee,ARA2,ARA2A2&A7A2,A2,10023,NaN,10023,100.0
1,2022-04-22,4,22,2022,1,5.450833,0,5.918333,NaN,46.7325,NaN,10010,0.67,0.61,Shasta,STA3,STA3A3,A3,10010,NaN,10010,100.0
2,2022-04-22,4,22,2022,1,5.450833,0,5.918333,NaN,46.7325,NaN,10385,0.67,0.61,Shasta,STA4,STA4A4,A4,10385,NaN,10385,100.0
3,2022-04-22,4,22,2022,1,5.450833,0,5.918333,NaN,46.7325,NaN,10595,0.67,0.61,Shasta,STA4,STA4A5,A5,10595,NaN,10595,100.0
4,2022-04-22,4,22,2022,1,5.450833,0,5.918333,NaN,46.7325,NaN,7806,0.67,0.61,Shasta,STA6,STA6A6,A6,7806,NaN,7806,100.0


### Use KNN for MISSING values for transparency


In [25]:
transparency_columns = ["AM Transparency", "PM Transparency"]
knn_imputer = KNNImputer(n_neighbors=5)
new_df.loc[:, transparency_columns] = knn_imputer.fit_transform(new_df[transparency_columns])

new_df.head()

,Date,Month,Day,Year,AM Feed,AM Transparency,PM Feed,PM Transparency,Stocked,Spring Temp (F),Morts,# fish,Dec Rain,Calmar Rain,Strain,Lot,Sub Lot,Raceway,# fish,Morts,Fish Alive,Fish survival rate
0,2022-04-22,4,22,2022,1,5.450833,0,5.918333,NaN,46.7325,NaN,10023,0.67,0.61,Arlee,ARA2,ARA2A2&A7A2,A2,10023,NaN,10023,100.0
1,2022-04-22,4,22,2022,1,5.450833,0,5.918333,NaN,46.7325,NaN,10010,0.67,0.61,Shasta,STA3,STA3A3,A3,10010,NaN,10010,100.0
2,2022-04-22,4,22,2022,1,5.450833,0,5.918333,NaN,46.7325,NaN,10385,0.67,0.61,Shasta,STA4,STA4A4,A4,10385,NaN,10385,100.0
3,2022-04-22,4,22,2022,1,5.450833,0,5.918333,NaN,46.7325,NaN,10595,0.67,0.61,Shasta,STA4,STA4A5,A5,10595,NaN,10595,100.0
4,2022-04-22,4,22,2022,1,5.450833,0,5.918333,NaN,46.7325,NaN,7806,0.67,0.61,Shasta,STA6,STA6A6,A6,7806,NaN,7806,100.0


### removed duplicate columns for One-hot vector encoding

In [ ]:
new_df = new_df.loc[:, ~new_df.columns.duplicated()]

In [ ]:
# Define categorical columns
categorical_columns = ["Stocked", "Morts", "Strain", "Lot", "Sub Lot", "Raceway"]

for col in categorical_columns:
    new_df[col] = new_df[col].astype(str).fillna("Unknown")

#OneHotEncoder
one_hot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

#trasnform
encoded_cats = one_hot_encoder.fit_transform(new_df[categorical_columns])

encoded_df = pd.DataFrame(encoded_cats, columns=one_hot_encoder.get_feature_names_out(categorical_columns))
new_df = new_df.drop(columns=categorical_columns)
new_df = pd.concat([new_df, encoded_df], axis=1)


new_df.head()

,Date,Month,Day,Year,AM Feed,AM Transparency,PM Feed,PM Transparency,Spring Temp (F),# fish,Dec Rain,Calmar Rain,Fish Alive,Fish survival rate,Stocked_N,Stocked_X,Stocked_Y,Stocked_nan,Morts_0.0,Morts_1.0,Morts_10.0,Morts_100.0,Morts_101.0,Morts_102.0,Morts_1028.0,Morts_104.0,Morts_105.0,Morts_106.0,Morts_107.0,Morts_1084.0,Morts_109.0,Morts_11.0,Morts_110.0,Morts_112.0,Morts_113.0,Morts_114.0,Morts_1150.0,Morts_1162.0,Morts_117.0,Morts_118.0,Morts_119.0,Morts_12.0,Morts_120.0,Morts_122.0,Morts_1228.0,Morts_123.0,Morts_126.0,Morts_127.0,Morts_13.0,Morts_130.0,Morts_131.0,Morts_132.0,Morts_134.0,Morts_135.0,Morts_136.0,Morts_139.0,Morts_14.0,Morts_140.0,Morts_146.0,Morts_148.0,Morts_15.0,Morts_150.0,Morts_151.0,Morts_152.0,Morts_1521.0,Morts_154.0,Morts_155.0,Morts_158.0,Morts_16.0,Morts_164.0,Morts_165.0,Morts_168.0,Morts_169.0,Morts_17.0,Morts_176.0,Morts_179.0,Morts_18.0,Morts_180.0,Morts_186.0,Morts_187.0,Morts_19.0,Morts_190.0,Morts_192.0,Morts_196.0,Morts_1963.0,Morts_198.0,Morts_2.0,Morts_20.0,Morts_202.0,Morts_205.0,Morts_206.0,Morts_21.0,Morts_214.0,Morts_22.0,Morts_2210.0,Morts_222.0,Morts_227.0,Morts_23.0,Morts_234.0,Morts_24.0,Morts_240.0,Morts_245.0,Morts_25.0,Morts_251.0,Morts_254.0,Morts_256.0,Morts_26.0,Morts_260.0,Morts_267.0,Morts_27.0,Morts_273.0,Morts_28.0,Morts_280.0,Morts_285.0,Morts_29.0,Morts_292.0,Morts_293.0,Morts_295.0,Morts_3.0,Morts_30.0,Morts_31.0,Morts_317.0,Morts_319.0,Morts_32.0,Morts_328.0,Morts_33.0,Morts_34.0,Morts_347.0,Morts_35.0,Morts_36.0,Morts_360.0,Morts_363.0,Morts_367.0,Morts_37.0,Morts_371.0,Morts_38.0,Morts_381.0,Morts_39.0,Morts_4.0,Morts_40.0,Morts_41.0,Morts_414.0,Morts_416.0,Morts_42.0,Morts_428.0,Morts_43.0,Morts_436.0,Morts_437.0,Morts_44.0,Morts_440.0,Morts_441.0,Morts_45.0,Morts_456.0,Morts_46.0,Morts_47.0,Morts_48.0,Morts_49.0,Morts_5.0,Morts_50.0,Morts_500.0,Morts_51.0,Morts_52.0,Morts_53.0,Morts_54.0,Morts_55.0,Morts_56.0,Morts_569.0,Morts_57.0,Morts_58.0,Morts_587.0,Morts_59.0,Morts_590.0,Morts_597.0,Morts_6.0,Morts_60.0,Morts_605.0,Morts_61.0,Morts_62.0,Morts_63.0,Morts_637.0,Morts_64.0,Morts_65.0,Morts_66.0,Morts_67.0,Morts_678.0,Morts_68.0,Morts_69.0,Morts_7.0,Morts_70.0,Morts_71.0,Morts_72.0,Morts_73.0,Morts_75.0,Morts_76.0,Morts_77.0,Morts_78.0,Morts_79.0,Morts_8.0,Morts_80.0,Morts_81.0,Morts_816.0,Morts_82.0,Morts_84.0,Morts_85.0,Morts_8533.0,Morts_86.0,Morts_87.0,Morts_88.0,Morts_89.0,Morts_9.0,Morts_90.0,Morts_907.0,Morts_91.0,Morts_94.0,Morts_96.0,Morts_98.0,Morts_nan,Strain_Arlee,Strain_Brook,Strain_Eagle Lake,Strain_Ennis,Strain_Geno,Strain_Hot Creek,Strain_Irwin X Arlee,Strain_Mix,Strain_Shasta,Strain_VAR,Lot_ARA1,Lot_ARA2,Lot_ARA5,Lot_ARA7,Lot_ARB7,Lot_ARC4,Lot_ARC8,Lot_Broods,Lot_BroodsB8,Lot_Brook,Lot_ELA5,Lot_ELC4,Lot_ENA1,Lot_ENA7,Lot_ENB6,Lot_ENB7,Lot_ENC3,Lot_ENC6,Lot_ENC7,Lot_GNA5,Lot_HCA2,Lot_HCA4,Lot_HCA6,Lot_HCB1,Lot_HCB2,Lot_HCB3,Lot_HCB5,Lot_HCC2,Lot_IAA1,Lot_IAA4,Lot_IAA5,Lot_IAA8,Lot_IAB1,Lot_IAB3,Lot_IAC2,Lot_IAC4,Lot_IAC6,Lot_Mix,Lot_STA2,Lot_STA3,Lot_STA4,Lot_STA6,Lot_STA7,Lot_STA8,Lot_STB1,Lot_STB2,Lot_STB4,Lot_STB6,Lot_STB7,Lot_STBB6,Lot_STBB7,Lot_STBC6,Lot_STBMB8,Lot_STC2,Lot_STC5,Sub Lot_ARA1,Sub Lot_ARA1A1,Sub Lot_ARA1A2,Sub Lot_ARA2,Sub Lot_ARA2&A7C3,Sub Lot_ARA2A2,Sub Lot_ARA2A2&A7A2,Sub Lot_ARA2B3,Sub Lot_ARA2B3&A7B3,Sub Lot_ARA5,Sub Lot_ARA5A5,Sub Lot_ARA5B5,Sub Lot_ARA7,Sub Lot_ARA7A7,Sub Lot_ARA7A8,Sub Lot_ARA7B8,Sub Lot_ARB7,Sub Lot_ARB7B7,Sub Lot_ARB7C7,Sub Lot_ARC4,Sub Lot_ARC4A1,Sub Lot_ARC4C4,Sub Lot_ARC4C4B4,Sub Lot_ARC4C4C4,Sub Lot_ARC8,Sub Lot_ARC8&B6C8,Sub Lot_BROODSB8P2,Sub Lot_BroodsB8,Sub Lot_BroodsP2P1,Sub Lot_BrookP1,Sub Lot_BrookP2,Sub Lot_ELA5,Sub Lot_ELC4,Sub Lot_ENA1,Sub Lot_ENA7,Sub Lot_ENB6,Sub Lot_ENB7,Sub Lot_ENC3,Sub Lot_ENC6,Sub Lot_ENC7,Sub Lot_GNA5,Sub Lot_HCA2,Sub Lot_HCA4,Sub Lot_HCA6,Sub Lot_HCB1,Sub Lot_HCB2,Sub Lot_HCB3,Sub Lot_HCB5,Sub Lot_HCC2,Sub Lot_IAA1,Sub Lot_IAA1&A8A5,Sub Lot_IAA1&A8A5C7,Sub Lot_IAA1A1,Sub Lot_IAA4,Sub Lot_IAA4A4,Sub Lot_IAA4A8,Sub Lot_IAA5,Sub Lot_IAA5A4,Sub Lot_IAA5A5,Sub L

#### Y will be Fish Survival Rate (%) = Formula: (fish alive / # fish) * 100
#### X will be the other values in the new_df. You can add or remove some valeus if you like.